In [1]:
import os
import json
import requests
from sqlalchemy import create_engine 
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
conn_string = f"postgresql://{os.getenv('DB__USER')}:{os.getenv('DB__PASSW')}@{os.getenv('LOCAL__HOST')}:{os.getenv('PG__PORT')}/{os.getenv('DB__NAME')}"

db = create_engine(conn_string) 
conn = db.connect() 

In [4]:
df = pd.DataFrame(
    [
        (1, 'SBOL', 1973453, 'DFGHrthfgfTJ', 1702339200000),
        (2, 'TINKOFF', 1412434, 'ergerRhdsht', 1702339276000),
        (3, 'ALFA', 2542352, 'ewtegHNTH', 1702339239000)
    ],
    columns = ['id', 'ticker', 'securityid', 'tradecode', 'datepost']
)
df.sample()

,id,ticker,securityid,tradecode,datepost
2,3,ALFA,2542352,ewtegHNTH,1702339239000


In [5]:
df.to_sql('quotes', conn, if_exists='append', index=False)

3

In [6]:
with open('connector_psql.json', 'r') as f:
    data = json.load(f)
data

{'name': 'movies-db-connector',
 'config': {'connector.class': 'io.debezium.connector.postgresql.PostgresConnector',
  'plugin.name': 'pgoutput',
  'tasks.max': '1',
  'database.hostname': '10.209.229.52',
  'database.port': '5432',
  'database.user': 'admin',
  'database.password': 'secret',
  'database.dbname': 'trades',
  'database.server.name': 'postgres',
  'table.include.list': 'public.quotes',
  'database.history.kafka.bootstrap.servers': 'broker:9092',
  'database.history.kafka.topic': 'schema-changes.quotes',
  'topic.prefix': 'uat.psql',
  'topic.creation.enable': 'true',
  'topic.creation.default.replication.factor': '1',
  'topic.creation.default.partitions': '1',
  'topic.creation.default.cleanup.policy': 'delete',
  'topic.creation.default.retention.ms': '604800000',
  'key.converter': 'io.confluent.connect.avro.AvroConverter',
  'value.converter': 'io.confluent.connect.avro.AvroConverter',
  'key.converter.schema.registry.url': 'http://10.209.229.52:8081',
  'value.conve

In [7]:
data['database.hostname'] = os.getenv('LOCAL__HOST')
data['database.port'] = os.getenv('PG__PORT')
data['database.user'] = os.getenv('DB__USER')
data['database.password'] = os.getenv('DB__PASSW')
data['key.converter.schema.registry.url'] = f"http://{os.getenv('LOCAL__HOST')}:{os.getenv('SCHEMA_REG__PORT')}"
data['value.converter.schema.registry.url'] = f"http://{os.getenv('LOCAL__HOST')}:{os.getenv('SCHEMA_REG__PORT')}"

In [8]:
url = f"http://{os.getenv('LOCAL__HOST')}:{os.getenv('DEBEZIUM__PORT')}/connectors"

In [ ]:
resp = requests.post(
    url = url, 
    headers = {'Content-Type':'application/json','Accept':'application/json'},
    json = data
)
resp.text